# Cryptography $CC4017$ -- Extra 11

## Q1: Demonstrating forgeability of plain RSA

A key property of digital signatures is that of existential unforgeability, which is expressed as a security game as follows.

**Existential Unforgeability**:

1 - We generate a keypair$sk, pk$

2 - We give $pk$ to the adversary, and allow him to request signatures of arbitrary messages. This means that the adversary is free to request signatures of anything, but every signed message is recorded.

3 - The adversary must then produce a pair $$m, s$$. He wins the game if 

i.$ $s$ is a valid signature for $m$

ii.$ the signature of $m$ was not requested in step 2.

**Plain RSA signature scheme** 

Plain RSA as a signature scheme sets $$e, n$$ as the public key and $$d, n$$ as the private key. Signatures of m are $m^{d}$ and validation of $$m, s$$ is checking if $m=s^{e}$.

It is easy to see that plain RSA is not existentially unforgeable. Indeed, anyone can produce forgery$1,1$, as $1^{e}= 1$ regardless of the value of $e$. However the issues of signing with plain RSA go aboveand beyond this singular case. Let’s go for a slightly more challenging scenario.

**Question:** 

Describe how an adversary can produce a valid forgery for the following experiment with probability 1.

**Existential unforgeability with a twist**

1 - We generate a keypair $$sk, pk$$ and select a bad value $b$.

2 - We give $pk$ and $b$ to the adversary, and allow him to request signatures of arbitrary messages. This means that the adversary is free to request signatures of anything, but every signed message is recorded.

3 - The adversary must then produce a signatures. He wins the game if 

i.$ $s$ is a valid signature for $b$

ii.$ the signature of $b$ was **not** requested in step 2.



## Answer

**Exploiting Plain RSA**

In plain RSA, the signature of a message $m$ is $s = m^d \mod n$, and verification involves checking if $m = s^e \mod n$.

Given the properties of RSA, the adversary can exploit the multiplicative property of RSA signatures. Specifically, if the adversary can find two messages $m_1$ and $m_2$ such that $m_1 \cdot m_2 \equiv b \mod n$, then the adversary can use the signatures of $m_1$ and $m_2$ to forge a signature for $b$.

**Request Signatures:**

The adversary requests the signatures for two messages $m_1$ and $m_2$ such that $m_1 \cdot m_2 \equiv b \mod n$.

Let $s_1 = m_1^d \mod n$ and $s_2 = m_2^d \mod n$ be the signatures for $m_1$ and $m_2$, respectively.

**Forge Signature:**

The adversary computes the forged signature $s$ for $b$ as $s = s_1 \cdot s_2 \mod n$.
Verification:

To verify the forged signature $s$, we check if $b = s^e \mod n$.

Since $s = s_1 \cdot s_2 \mod n$, we have: 

$s^e \equiv (s_1 \cdot s_2)^e \mod n \equiv s_1^e \cdot s_2^e \mod n \equiv m_1 \cdot m_2 \mod n \equiv b \mod n $

Therefore, $s$ is a valid signature for $b$.

By exploiting the multiplicative property of RSA, the adversary can produce a valid forgery for the message (b) with probability 1.